# American Sign Language - Convoltion Neural Network - Using Tensorflow and Keras

**Import Necessary Libraries**

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

import matplotlib.pyplot as plt
import seaborn as sns
from IPython import display
from IPython.display import clear_output

import glob
import imageio
import time

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# ASL
**Image Grid**

 Below is a grid that displays all sign language representations of each letter of the alphabet from (A-Z)

In [ ]:
from IPython.display import Image
Image(filename='/kaggle/input/sign-language-mnist/american_sign_language.PNG') 

# Data Exploration

The dataset format is patterned to match closely with the classic MNIST. Each training and test case represents a label (0-25) as a one-to-one map for each alphabetic letter A-Z (and no cases for 9=J or 25=Z because of gesture motions). The training data (27,455 cases) and test data (7172 cases) are approximately half the size of the standard MNIST handwritten digit dataset but otherwise similar with a header row of label, pixel1,pixel2….pixel784 which represent a single 28x28 pixel image with grayscale values between 0-255.



In [ ]:
train_df = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
test_df = pd.read_csv("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")
y       = test_df['label']

In [ ]:
validation_df = test_df.sample(900)

In [ ]:
train_df.head()

From the dataset above, each row is a representation of an instance of a 28 X 28 image and a label to denote which letter the 28x28 image represents.

We first need to split out data into X and y such that X will only hold the respective pixel values of each image while y will be simply be the one-hot encoded version of our labels. We will then need to check for presence of any data imbalance which will bring lots of problems when training our cnn model

#  Data Splitting

In [ ]:
X_train = train_df.drop("label",axis = 1)
y_train = train_df["label"]

X_test = test_df.drop("label",axis = 1)
y_test = test_df["label"]

X_valid = validation_df.drop("label",axis = 1)
y_valid = validation_df["label"]




del train_df
del test_df
del validation_df

In [ ]:
X_train.head()

In [ ]:
plt.figure(figsize = (20,10)) # Label Count
sns.set_style("darkgrid")
sns.countplot(y_train )

**DUE TO THE INABILITY TO CAPTURE MOTION IN A PICTURE, THE LETTERS "J" DENOTED BY 9 AND Z DENOTED BY "25" WERE NOT PRESENT IN THE DATASET**

In [ ]:
y_train.value_counts()

The training set seems to evenly/ unifromly distributed  with a ratio of approximately 1:1 between a pair of the possible categories being maintained

**One hot encoding**

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(np.array(y_train).reshape(-1,1))

In [ ]:
y_train = enc.transform(np.array(y_train).reshape(-1,1)).toarray()
y_test = enc.transform(np.array(y_test).reshape(-1,1)).toarray()
y_valid = enc.transform(np.array(y_valid).reshape(-1,1)).toarray()


In [ ]:
y_train.shape

In [ ]:
y_valid.shape

In [ ]:
X_train = X_train.values
X_test = X_test.values
X_valid = X_valid.values

# Image pre-processing
**Image normalization**

In [ ]:
X_train = X_train / 255
X_test =  X_test / 255
X_valid =  X_valid / 255

**Image Reshape**

In [ ]:
X_train = X_train.reshape(-1,28,28,1)
X_test =  X_test.reshape(-1,28,28,1)
X_valid =  X_valid.reshape(-1,28,28,1)

In [ ]:
plt.figure(figsize=(10,10))
for i in range(1,10):
  plt.subplot(3,3,i)
  random_num = np.random.randint(0,len(X_train))
  plt.imshow(X_train[random_num][:,:,:], cmap = "gray")
  plt.grid(False)
  plt.axis('off')

# Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
                          rotation_range = 0,
                          height_shift_range=0.2,
                          width_shift_range=0.2,
                          shear_range=0,
                          zoom_range=0.2,
                          horizontal_flip=True,
                          fill_mode='nearest')
datagen.fit(X_train)

In [ ]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=1, mode='auto',
        restore_best_weights=True)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 5, verbose=1,factor=0.3, mode="max")
callback_list = [monitor, learning_rate_reduction]

In [ ]:
model = Sequential()

model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu', input_shape = (28,28,1)))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Dropout(0.1))
model.add(BatchNormalization())


model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(BatchNormalization())


model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))

model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy',tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')])
model.summary()

In [ ]:

history = model.fit(datagen.flow(X_train, y_train, batch_size = 128) ,epochs = 100 , \
                    validation_data = (X_test,y_test) ,callbacks = callback_list)

# Model Investigation

In [ ]:
def plot_validation_curves(result):
  result = pd.DataFrame(result)
  fig, axs = plt.subplots(1,2)
  result[['loss','val_loss']].plot(figsize=(10, 3),ax=axs[0])
  axs[0].set_title('Train vs validation Loss')
  axs[0].set_xlabel('Epochs')
  axs[0].set_ylabel('Loss')
  result[['accuracy','val_accuracy']].plot(figsize=(10, 3),ax=axs[1])
  axs[1].set_title('Train vs validation Accuracy')
  axs[1].set_xlabel('Epochs')
  axs[1].set_ylabel('Accuracy')

In [ ]:
plot_validation_curves(history.history)

In [ ]:
X_test.shape

In [ ]:
print("Accuracy of the model is - " , model.evaluate(X_test,y_test)[1]*100 , "%")

**Confusion Matrix**

Since there was no class 9 in the dataset, we will have to manipulate the predictions by adding one (1) to all class numerals that are greater than 8 such that instead of ***0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23*** we will now have ***0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24***

In [ ]:

predictions = model.predict_classes(X_test)
for i in range(len(predictions)):
    if(predictions[i] >= 9):
        predictions[i] += 1
predictions[:5]     

In [ ]:
classes = ["Class " + str(i) for i in range(25) if i != 9]
print(classification_report(y, predictions, target_names = classes))

In [ ]:
cm = confusion_matrix(y,predictions)
cm = pd.DataFrame(cm , index = [i for i in range(25) if i != 9] , columns = [i for i in range(25) if i != 9])

plt.figure(figsize = (15,15))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='')

**Some sample predictions **

In [ ]:


i = 0
for c in range(0,4):
    
    index = np.random.randint(0,len(X_test))
    plt.subplot(2,2,i+1)
    plt.imshow(X_test[index].reshape(28,28), cmap="gray")
    plt.title("Pred {} -Actual {}".format(predictions[index], y[index]))
 
    i += 1

In [ ]:
model.save_weights('hand_gesture_model.h5')